In [20]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"
!pip install ipywidgets
!pip install pycountry geonamescache

In [21]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [22]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
OPENWEATHER_API_KEY = UserSecretsClient().get_secret("OPENWEATHER_API_KEY")

In [23]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [24]:
import requests

OPENWEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

def get_weather(location: str) -> dict:
    """Fetch current weather data for a given location using OpenWeatherMap API."""
    params = {
        "q": location,
        "appid": OPENWEATHER_API_KEY,
        "units": "metric"  # Change to 'imperial' for Fahrenheit
    }
    try:
        response = requests.get(OPENWEATHER_URL, params=params)
        response.raise_for_status()
        data = response.json()

        weather_info = {
            "location": data.get("name"),
            "temperature": data["main"]["temp"],
            "feels_like": data["main"]["feels_like"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"],
            "description": data["weather"][0]["description"].capitalize()
        }
        return weather_info

    except requests.RequestException as e:
        return {"error": str(e)}

In [25]:
def get_weather_forecast(location: str, hours_ahead: int) -> dict:
    """
    Retrieves the forecasted weather for a specified city and number of hours into the future.

    Args:
        location (str): Name of the city, town, or place (e.g., "Vancouver", "Toronto").
        hours_ahead (int): How many hours from now to get the forecast (e.g., 6, 12, 24).

    Returns:
        dict: Weather information such as temperature, humidity, wind speed, and a short description.
    """
    api_key = OPENWEATHER_API_KEY  # Preloaded from Kaggle secrets

    # Fallback in case Gemini doesn't send the value (optional)
    if hours_ahead is None:
        hours_ahead = 24

    params = {
        "q": location,
        "appid": api_key,
        "units": "metric"
    }

    try:
        response = requests.get(OPENWEATHER_FORECAST_URL, params=params)
        response.raise_for_status()
        data = response.json()

        idx = hours_ahead // 3  # 3-hour intervals
        forecast_entry = data["list"][idx]

        return {
            "forecast_time": forecast_entry["dt_txt"],
            "location": data["city"]["name"],
            "temperature": forecast_entry["main"]["temp"],
            "feels_like": forecast_entry["main"]["feels_like"],
            "humidity": forecast_entry["main"]["humidity"],
            "wind_speed": forecast_entry["wind"]["speed"],
            "description": forecast_entry["weather"][0]["description"].capitalize()
        }

    except requests.RequestException as e:
        return {"error": str(e)}

In [28]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

instruction = """You are a helpful assistant that recommends outfits based on future weather forecasts.

Before recommending an outfit, use the `get_weather_forecast` tool to check the forecast for the location and time requested by the user.

Ask the user for both:
- The location (city/town)
- The number of hours into the future they want the forecast for

Then use that weather information to suggest what to wear."""

chat = client.chats.create(
    # model="gemini-1.5-flash",  # Or "gemini-2.0-flash"
    model="gemini-2.0-flash", 
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        # tools=[get_weather_forecast],  # Registers the tool
        tools=[get_weather],  # Registers the tool
    )
)

In [29]:
response = chat.send_message("What should I wear in Chongqing in 12 hours later?")
print(response.text)

OK. The weather in Chongqing in 12 hours is forecast to be: Overcast clouds, with a temperature of 14.94 degrees Celsius, feels like 14.74 degrees Celsius, humidity of 86%, and wind speed of 0.64 m/s.

I would recommend wearing a sweater or light jacket with long pants. You might also want to bring an umbrella or raincoat in case of rain.



In [30]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import geonamescache

# --- Load city data ---
gc = geonamescache.GeonamesCache()
cities_dict = gc.get_cities()
major_cities = sorted(set(city["name"] for city in cities_dict.values()))

# --- Unified input width and label style ---
label_width = "80px"
input_width = "300px"

# --- Widgets ---
city_dropdown = widgets.Combobox(
    options=major_cities,
    placeholder="Start typing a city...",
    description="City:",
    ensure_option=False,
    style={"description_width": label_width},
    layout=widgets.Layout(width=input_width)
)

custom_city_input = widgets.Text(
    placeholder="Or type another city",
    description="Custom:",
    style={"description_width": label_width},
    layout=widgets.Layout(width=input_width)
)

hours_slider = widgets.IntSlider(
    value=24,
    min=0,
    max=72,
    step=1,
    description='Hours:',
    style={"description_width": label_width},
    layout=widgets.Layout(width=input_width)
)

go_button = widgets.Button(
    description="Get Recommendation",
    button_style="success",
    layout=widgets.Layout(width=input_width)
)

output = widgets.Output()

# --- Button logic ---
def on_go_clicked(b):
    output.clear_output()
    location = custom_city_input.value.strip() or city_dropdown.value.strip()
    hours = hours_slider.value

    if not location:
        with output:
            print("⚠️ Please enter or select a city.")
        return

    with output:
        print(f"📍 Location: {location}")
        print(f"⏳ Forecast for {hours} hours later...\n")
        response = chat.send_message(f"What should I wear in {location} in {hours} hours?")
        print(response.text)

go_button.on_click(on_go_clicked)

# --- Display ---
form = widgets.VBox([
    city_dropdown,
    custom_city_input,
    hours_slider,
    go_button,
    output
])
display(form)